### __Chatbot__

Below I use the chat format to provide the conversations between user and ChatGPT, that are specialized for different tasks and situations.

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Try the custom 'messages' list of dicts to provide ChatGPT the context and specific settings of the conversation.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

Run the response function with a high level of temperature to get more various and diverse answers.

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

```
To get to the other side, my lord!
```

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Dima'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

```
Hello Dima! I'm happy to chat with you. How are you doing today?
```

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

```
I apologize, as an AI language model, I do not have access to your personal details or information. Can you please tell me your name?
```

Because each conversation with ChatGPT is a standalone iteraction, one must provide the context to the model so that it could correctly continue the communication. 

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Dima'},
{'role':'assistant', 'content': "Hello Dima, it's nice to meet you! \
How can I assist you today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

```
Your name is Dima!
```

### __Chat Bot for surveys__

Below I provide the code of the chat bot which can conduct a simple survey with yes/no questions. Depending on the survey target, it can be elaborated to ask questions with several answer options or open questions.

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


The list of questions that can be changed depending on the subject of the survey.

In [ ]:
questions = ['Did you feel happiness yesterday?', 'Do you have the permanent job now?',
'Do you have minor children?', 'Do you estimate your income as above average \
(yes - above average, no - below average)?']

Writing an instruction for ChatGPT describing survey process and technical details.

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':f"""
You are SurveyBot, an automated service to collect public opinion on different topics.
You first greet the participant, then ask whether he or she wants to share \
hir or her opinion anonymously.
If the participant doesn't agree, then thank him/her and end the coversation.
If he/she agrees, then ask the yes/no questions from the list provided: \
'''{questions}'''
Ask only 1 question at a time.
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Getting an output of the survey, converted to 0/1 format, making them more suitable for further analysis.

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content': """"
create a json summary of the previous survey. \
provide the output as JSON file with the sequence number \
of the question as the key and the corresponding value as value. \
Before that convert the answers to 0/1 format, where 0 is 'no' and 1 is 'yes'. \
If it's impossible to clearly transform, assign the 'null' value.
Provide only JSON file in your answer."""},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

```
{
    "1": 1,
    "2": 0,
    "3": 0,
    "4": 0
}
```